In [32]:
import json
import csv
import requests
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import WazeRouteCalculator
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [3]:
weather_data = {}

for i in range(3,10):
    url = "http://api.apixu.com/v1/history.json?key=3d155e6fb2b3462583b90831190907&q=rg1&dt=2019-07-"+str(i)
    response = requests.get(url)
    data = response.json()
    weather_data[i] = data



In [4]:
with open('weather_data.txt', 'w') as outfile:
    json.dump(weather_data, outfile)

In [11]:
url = "https://rtl2.ods-live.co.uk//api/busstops?key=hvOtkiqAwK"
response = requests.get(url)
stop_list_data = response.json()

with open('stop_list.txt', 'w') as outfile:
    json.dump(stop_list_data, outfile)

In [20]:
url = "https://rtl2.ods-live.co.uk//api/linePatterns?key=hvOtkiqAwK&service=17"
response = requests.get(url)
route_17_stops = response.json()

with open('route_17_stops.txt', 'w') as outfile:
    json.dump(route_17_stops, outfile)

In [5]:
weather = {}

for i in range(3,10):
    url = "http://api.apixu.com/v1/history.json?key=3d155e6fb2b3462583b90831190907&q=rg1&dt=2019-07-"+str(i)
    response = requests.get(url)
    data = response.json()
    day = data["forecast"]["forecastday"][0]["hour"]
    days = {}
    for j in range(0,24):
        days[j] = [day[j]["condition"]["text"], day[j]["precip_mm"], day[j]["precip_in"], 
                   day[j]["humidity"]]
    weather[i] = days

In [26]:
weather

{3: {0: [u'Partly cloudy', 0.0, 0.0, 91],
  1: [u'Partly cloudy', 0.0, 0.0, 90],
  2: [u'Partly cloudy', 0.0, 0.0, 90],
  3: [u'Partly cloudy', 0.0, 0.0, 90],
  4: [u'Partly cloudy', 0.0, 0.0, 89],
  5: [u'Partly cloudy', 0.0, 0.0, 88],
  6: [u'Partly cloudy', 0.0, 0.0, 87],
  7: [u'Partly cloudy', 0.0, 0.0, 81],
  8: [u'Partly cloudy', 0.0, 0.0, 75],
  9: [u'Partly cloudy', 0.0, 0.0, 69],
  10: [u'Partly cloudy', 0.0, 0.0, 66],
  11: [u'Patchy rain possible', 0.03, 0.0, 64],
  12: [u'Patchy rain possible', 0.04, 0.0, 61],
  13: [u'Patchy rain possible', 0.02, 0.0, 61],
  14: [u'Partly cloudy', 0.0, 0.0, 62],
  15: [u'Partly cloudy', 0.0, 0.0, 62],
  16: [u'Partly cloudy', 0.0, 0.0, 66],
  17: [u'Partly cloudy', 0.0, 0.0, 69],
  18: [u'Partly cloudy', 0.0, 0.0, 72],
  19: [u'Partly cloudy', 0.0, 0.0, 78],
  20: [u'Partly cloudy', 0.0, 0.0, 83],
  21: [u'Partly cloudy', 0.0, 0.0, 89],
  22: [u'Partly cloudy', 0.0, 0.0, 89],
  23: [u'Clear', 0.0, 0.0, 90]},
 4: {0: [u'Clear', 0.0, 0.0, 9

In [6]:
with open('weather.txt', 'w') as outfile:
    json.dump(weather, outfile)

In [39]:
dataset_17 = []

for i in range(3,10):
    url = "https://rtl2.ods-live.co.uk/api/trackingHistory?key=hvOtkiqAwK&service=17&date=2019-07-"+str(i)+"&vehicle=707&location="
    response = requests.get(url)
    data = response.json()
    day = []
    for track in data:
        if track["ArrivalTime"] != "":
            arrival_time = time.strptime(track["ScheduledArrivalTime"], "%Y-%m-%d %H:%M:%S")
            date = str(arrival_time.tm_year)+"-"+str(arrival_time.tm_mon)+"-"+str(arrival_time.tm_mday)
            
            weather_cond = weather[arrival_time.tm_mday][arrival_time.tm_hour][0]
            precip_mm = weather[arrival_time.tm_mday][arrival_time.tm_hour][1]
            precip_in = weather[arrival_time.tm_mday][arrival_time.tm_hour][2]
            humidity = weather[arrival_time.tm_mday][arrival_time.tm_hour][3]
                    
            stop = [obj for obj in stop_list_data if obj['location_code']==track["LocationCode"]][0]
            lat = stop['latitude']
            long = stop['longitude']     
            
            stop_ind = [i for i, obj in enumerate(route_17_stops) if obj['location_code']==track["LocationCode"]][0]
            next_stop = route_17_stops[(stop_ind+1)%len(route_17_stops)]
            next_stop_name = next_stop['location_name']
            next_stop_code = next_stop['location_code']
            next_stop_from_list = [obj for obj in stop_list_data if obj['location_code']==next_stop_code][0]
            next_lat = next_stop_from_list['latitude']
            next_long = next_stop_from_list['longitude']
            
            from_address = str(lat)+','+str(long)
            to_address = str(next_lat)+','+str(next_long)
            route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, 'EU', log_lvl=None).calc_route_info()
            stops_distance = route[1]
            normal_time = route[0]
            
            real_arrival = time.strptime(track["ArrivalTime"], "%Y-%m-%d %H:%M:%S")

            punctuality = time.mktime(real_arrival)-time.mktime(arrival_time)
            row = [date, track["ScheduledArrivalTime"], track["ScheduledDepartureTime"], 
                   track["ArrivalTime"], track["DepartureTime"], track["LineRef"], track["LocationName"], 
                   track["LocationCode"], lat, long, next_stop_name, next_stop_code, next_lat, next_long,
                   stops_distance, normal_time, weather_cond, precip_mm, precip_in, humidity, punctuality]
            dataset_17.append(row)

ConnectionError: HTTPSConnectionPool(host='rtl2.ods-live.co.uk', port=443): Max retries exceeded with url: /api/trackingHistory?key=hvOtkiqAwK&service=17&date=2019-07-3&vehicle=707&location= (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11b385110>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [17]:
dataset_17

[['2019-7-3',
  u'2019-07-03 07:16:00',
  u'2019-07-03 07:16:00',
  u'2019-07-03 07:12:48',
  u'2019-07-03 07:16:42',
  u'17',
  u'Friar Street stop FN',
  u'039026100006',
  u'51.456805',
  u'-0.972627',
  u'Partly cloudy',
  0.0,
  0.0,
  81,
  -192.0],
 ['2019-7-3',
  u'2019-07-03 07:17:00',
  u'2019-07-03 07:17:00',
  u'2019-07-03 07:17:12',
  u'2019-07-03 07:17:27',
  u'17',
  u'Blagrave Street Stop EO',
  u'039028150002',
  u'51.457632',
  u'-0.970378',
  u'Partly cloudy',
  0.0,
  0.0,
  81,
  12.0],
 ['2019-7-3',
  u'2019-07-03 07:20:00',
  u'2019-07-03 07:20:00',
  u'2019-07-03 07:18:35',
  u'2019-07-03 07:18:54',
  u'17',
  u'Kings Road Stop ME',
  u'039025920002',
  u'51.455018',
  u'-0.968723',
  u'Partly cloudy',
  0.0,
  0.0,
  81,
  -85.0],
 ['2019-7-3',
  u'2019-07-03 07:22:00',
  u'2019-07-03 07:22:00',
  u'2019-07-03 07:20:41',
  u'2019-07-03 07:20:51',
  u'17',
  u'Huntley and Palmers Stop 4',
  u'039026410004',
  u'51.455073',
  u'-0.961713',
  u'Partly cloudy',
  0

In [24]:
header = ["date", "scheduled_arrival_time", "scheduled_departure_time", "arrival_time", "departure_time"
         , "service_num", "location_name", "location_code", "latitude", "longitude", "next_stop_name", 
          "next_stop_code", "next_lat", "next_long", "normal_time", "weather_cond", "preci_mm", "precip_in", 
          "humidity","punctuality"]

In [25]:
dataset_17_df = pd.DataFrame(dataset_17, columns=header)
dataset_17_df

,date,scheduled_arrival_time,scheduled_departure_time,arrival_time,departure_time,service_num,location_name,location_code,latitude,longitude,next_stop_name,next_stop_code,next_lat,next_long,weather_cond,preci_mm,precip_in,humidity,punctuality
0,2019-7-3,2019-07-03 07:16:00,2019-07-03 07:16:00,2019-07-03 07:12:48,2019-07-03 07:16:42,17,Friar Street stop FN,039026100006,51.456805,-0.972627,Blagrave Street Stop EO,039028150002,51.457632,-0.970378,Partly cloudy,0.0,0.0,81,-192.0
1,2019-7-3,2019-07-03 07:17:00,2019-07-03 07:17:00,2019-07-03 07:17:12,2019-07-03 07:17:27,17,Blagrave Street Stop EO,039028150002,51.457632,-0.970378,Friar Street stop FP,039028170004,51.456718,-0.972960,Partly cloudy,0.0,0.0,81,12.0
2,2019-7-3,2019-07-03 07:20:00,2019-07-03 07:20:00,2019-07-03 07:18:35,2019-07-03 07:18:54,17,Kings Road Stop ME,039025920002,51.455018,-0.968723,Huntley and Palmers Stop 4,039026410004,51.455073,-0.961713,Partly cloudy,0.0,0.0,81,-85.0
3,2019-7-3,2019-07-03 07:22:00,2019-07-03 07:22:00,2019-07-03 07:20:41,2019-07-03 07:20:51,17,Huntley and Palmers Stop 4,039026410004,51.455073,-0.961713,Eldon Road opp,039025980002,51.453855,-0.957252,Partly cloudy,0.0,0.0,81,-79.0
4,2019-7-3,2019-07-03 07:23:00,2019-07-03 07:23:00,2019-07-03 07:21:32,2019-07-03 07:21:57,17,Eldon Road opp,039025980002,51.453855,-0.957252,Reading College adj,039027450002,51.453453,-0.953460,Partly cloudy,0.0,0.0,81,-88.0
5,2019-7-3,2019-07-03 07:24:00,2019-07-03 07:24:00,2019-07-03 07:22:24,2019-07-03 07:22:50,17,Reading College adj,039027450002,51.453453,-0.953460,Cemetery Junction Stop E1,039025480002,51.453147,-0.950345,Partly cloudy,0.0,0.0,81,-96.0
6,2019-7-3,2019-07-03 07:25:00,2019-07-03 07:25:00,2019-07-03 07:23:08,2019-07-03 07:23:25,17,Cemetery Junction Stop E1,039025480002,51.453147,-0.950345,Hamilton Road opp,039026240002,51.452358,-0.945743,Partly cloudy,0.0,0.0,81,-112.0
7,2019-7-3,2019-07-03 07:27:00,2019-07-03 07:27:00,2019-07-03 07:24:47,2019-07-03 07:24:54,17,Hamilton Road opp,039026240002,51.452358,-0.945743,College Road opp,039025690002,51.451490,-0.942107,Partly cloudy,0.0,0.0,81,-133.0
8,2019-7-3,2019-07-03 07:28:00,2019-07-03 07:28:00,2019-07-03 07:25:18,2019-07-03 07:25:27,17,College Road opp,039025690002,51.451490,-0.942107,Palmer Park Avenue adj,039026890002,51.449340,-0.938027,Partly cloudy,0.0,0.0,81,-162.0
9,2019-7-3,2019-07-03 07:29:00,2019-07-03 07:29:00,2019-07-03 07:27:05,2019-07-03 07:27:24,17,Palmer Park Avenue adj,039026890002,51.449340,-0.938027,St Peters Road adj,039027350002,51.447332,-0.935627,Partly cloudy,0.0,0.0,81,-115.0


In [26]:
dataset_17_df.to_csv('route_17.csv')

In [28]:
import WazeRouteCalculator

from_address = '51.456805, -0.972627'
to_address = '51.457632, -0.970378'
region = 'EU'
route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region)
route.calc_route_info()

From: 51.456805, -0.972627 - to: 51.457632, -0.970378
Time 1.03 minutes, distance 0.32 km.


(1.0333333333333334, 0.319)